In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook as tqdm

import torch
import tensorflow as tf
from transformers import T5Tokenizer, AutoTokenizer,MT5Tokenizer
from transformers import MT5ForConditionalGeneration, AdamW

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

# pl.seed_everything(42)
tf.random.set_seed(42)
import warnings
warnings.filterwarnings("ignore")

In [19]:


MODEL_NAME = "google/mt5-base"
tokenizer_name_or_path="google/mt5-base"
tokenizer = MT5Tokenizer.from_pretrained(MODEL_NAME, model_max_length= INPUT_MAX_LEN)

In [22]:
import joblib

In [23]:
# model=joblib.load('..\mt5base_final_model.joblib')
model=joblib.load('mt5base_final_model.joblib')

In [24]:
def generate_paraphrase(question1):

    inputs_encoding =  tokenizer(
        question1,
        add_special_tokens=True,
        max_length= 256,
        padding = 'max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors="pt" 
        )


    generate_ids = model.model.generate(
        input_ids = inputs_encoding["input_ids"],
        attention_mask = inputs_encoding["attention_mask"],
        do_sample=True,
        max_length=64,
        top_k=40,
        top_p=0.98,
        early_stopping=True,
        num_return_sequences = 1,
        no_repeat_ngram_size=2,
        )

    preds = [
        tokenizer.decode(gen_id,
        skip_special_tokens=True, 
        clean_up_tokenization_spaces=True)
        for gen_id in generate_ids
    ]

    return "".join(preds)

In [25]:
text='उनीहरूको सेवा सुविधा भने रोकिएको थियो ।'
print(generate_paraphrase(text))

उनीहरूको सेवा सुविधा रोकिएको छ।


In [26]:
text='क्रस र प्रेम बीचको भिन्नता कसरी बताउनुहुन्छ?'
print(generate_paraphrase(text))

तपाईंले क्रस र प्रेमको भिन्नता कसरी भन्नुहुन्छ?


In [ ]:
train_model = MT5Model.load_from_checkpoint(dir_to_ckpt)